# Similarity Comparison for Tests

## Fuzzy string matching

In [ ]:
# TODO

## Sentence Embeddings

In [3]:

from sentence_transformers import SentenceTransformer

# The "all-MiniLM-L6-v2" model is used for demonstration
model = SentenceTransformer("all-MiniLM-L6-v2")

expected = ["Rangieren", "420", "Gleis", "Abstellgleis", "auf Sicht fahren", "Signal"]
actual = ["Rangieren", "420er", "Gleis", "Abstellgleis", "auf Sicht fahren", "Signal"]

sentences = expected + actual

embeddings = model.encode(sentences)

similarities = model.similarity(embeddings, embeddings)
for i in range(len(expected)):
    print(f"Expected: {expected[i]}, Actual: {actual[i]}, Similarity: {similarities[i][i+len(expected)]}")

Expected: Rangieren, Actual: Rangieren, Similarity: 1.0
Expected: 420, Actual: 420er, Similarity: 0.8273268938064575
Expected: Gleis, Actual: Gleis, Similarity: 1.0000003576278687
Expected: Abstellgleis, Actual: Abstellgleis, Similarity: 1.000000238418579
Expected: auf Sicht fahren, Actual: auf Sicht fahren, Similarity: 1.0
Expected: Signal, Actual: Signal, Similarity: 1.0


## Assesment using LLMs

In [49]:
import random
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()
input_text = "Rangiere mir bitte mal den 420er von Gleis 3 auf das Abstellgleis. Passt auf, du musst auf Sicht bis zu den Signalen fahren."
expected_terms = ["Rangieren", "420", "Abstellgleis", "auf Sicht fahren", "Signal"]
actual_terms =   ["Rangieren", "420er", "Abstellgleis", "auf Sicht fahren", "Signal"]

def run_test(shuffle=False):
    expected = expected_terms.copy()
    actual = actual_terms.copy()
    if shuffle:
        random.shuffle(expected)
        random.shuffle(actual)
        print(f"Expected: {expected}, Actual: {actual}")
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        response_format={"type": "text"},
        temperature=0,
        top_p=0,
        messages=[
            {
                "role": "user",
                "content": "Bewerte die Ähnlichkeit der Ergebnisse der Term Extraktion. Gegeben ist ein Ausgangstext, "
                     "aus dem Fachbegriffe extrahiert werden mussten. Der Text ist gegeben. Darunter stehen die erwarteten Begriffe, "
                     "die extrahiert werden sollten. Zum Schluss stehen die tatsächlich extrahierten Begriffe. "
                     "Bewerte die Ähnlichkeit der extrahierten Begriffe. "
                     "Ignoriere die Reihenfolge der Begriffe."
                     "Sobald sich ein Begriff grundlegend unterscheidet, beende sofort mit FALSE."
                     "Wenn ein erwarteter Begriff gänzlich fehlt, beende sofort mit FALSE."
                     "Wenn ein Begriff extrahiert wurde, der sicher kein Fachbegriff ist, beende sofort mit FALSE."
                     "Ansonsten Ende sofort mit TRUE."
                     "Bewerte die extrahierten Begriffe."
            },
            {
                "role": "user",
                "content": f"""{input_text}\n\nErwartete Begriffe: {", ".join(expected)}\n\nTatsächliche Begriffe: {", ".join(actual)}"""
            }
        ],
        logprobs=True,
        seed=42,
        store=False,
        top_logprobs=5,
        frequency_penalty=0,
        presence_penalty=0,
    )
    result, logprobs = response.choices[0].message.content, response.choices[0].logprobs
    # Get probabilities for TRUE and FALSE as the last token
    probs = {token.token: float(np.exp(token.logprob)) for token in logprobs.content[0].top_logprobs if token.token == "TRUE" or token.token == "FALSE"}
    # Normalize the probabilities to only account for TRUE and FALSE
    # This might actually distort the result, as other, maybe more likely tokens are ignored (however, such results can be considered faulty)
    total_end = sum(probs.values())
    normalized_probs = {token: value / total_end for token, value in probs.items()}
    print(f"Probability for success of test {normalized_probs['TRUE']}")

In [50]:
for i in range(5):
    run_test(shuffle=False)

Probability for success of test 0.2942149597859341
Probability for success of test 0.46879062662624377
Probability for success of test 0.2942149597859341
Probability for success of test 0.26894142136999516
Probability for success of test 0.24508501864634824


In [51]:
for i in range(5):
    run_test(shuffle=True)

Expected: ['420', 'auf Sicht fahren', 'Rangieren', 'Abstellgleis', 'Signal'], Actual: ['Abstellgleis', 'Rangieren', 'Signal', '420er', 'auf Sicht fahren']
Probability for success of test 0.2689414096510109
Expected: ['Rangieren', 'auf Sicht fahren', '420', 'Signal', 'Abstellgleis'], Actual: ['Rangieren', '420er', 'auf Sicht fahren', 'Abstellgleis', 'Signal']
Probability for success of test 0.053403330553099
Expected: ['Rangieren', 'auf Sicht fahren', 'Signal', '420', 'Abstellgleis'], Actual: ['Signal', '420er', 'Abstellgleis', 'Rangieren', 'auf Sicht fahren']
Probability for success of test 0.9820137910906878
Expected: ['Abstellgleis', '420', 'auf Sicht fahren', 'Signal', 'Rangieren'], Actual: ['420er', 'auf Sicht fahren', 'Signal', 'Abstellgleis', 'Rangieren']
Probability for success of test 0.8807970779778824
Expected: ['Rangieren', 'Signal', '420', 'auf Sicht fahren', 'Abstellgleis'], Actual: ['Rangieren', 'auf Sicht fahren', 'Signal', '420er', 'Abstellgleis']
Probability for succes